# 初期セットアップを行う

ワークフロー実行のための準備を行います。
上から順番にセルを実行してください。

## １．ユーザーの認証を行う

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってデータガバナンス機能に登録したユーザー名、パスワード、メールアドレスを入力してください。

◆◆◆開発メモ◆◆◆  
ゆくゆくはSSOに対応し、コンテナ立ち上げ毎のユーザー名、パスワード、メールアドレスの入力を避けたい

In [ ]:
import json
from scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

tokens = []
access_token = {}

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    email = input("メールアドレス：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    if response.status_code == HTTPStatus.UNAUTHORIZED:
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")
        continue
    
    tokens = response.json()
    if len(tokens) >= 1:
        access_token = response.json()[-1]
        clear_output()
        break
    elif len(tokens) < 1:
        # 既存のトークンがなければ作成する
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))
        if response.status_code == HTTPStatus.CREATED:
            access_token = response.json()
            clear_output()
            break
    
!git config --global user.name $name
!git config --global user.email $email
!cp ~/.gitconfig ~/WORKFLOW/PACKAGE/.gitconfig

print("認証が正常に完了しました。次の手順へお進みください。")

## 2. データ同期のための設定をする

この手順では、今の実行環境とデータガバナンス機能のリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%bash
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi
if [ ! -d ~/.datalad/ ]; then
    # Dataladのデータセットでなければデータセット化する
    datalad create --force /home/jovyan
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json
from scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/keys?token=' + access_token['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

# コンテナを消す際にコンテナとつなぐための公開鍵も削除のため、
# パラメータとしてGINから発行された鍵IDを保存
if response.status_code == HTTPStatus.CREATED:
    # params.jsonへの追記（鍵ID）
    params['ginKeyId'] = str(response.json()['id'])
    with open(utils.fetch_param_file_path(), mode='w') as f:
        json.dump(params, f, indent=4)
    print('Public key is ready.')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('Public key is ready before time.')

In [ ]:
# sibling url をsshに変更する
import os
import tempfile
from datalad import api
from IPython.display import clear_output

remote_info = 'remote.txt'

with tempfile.TemporaryDirectory() as tmpdir:
    # 使い終わったらPython側で消してくれるTemporaryDirectoryを利用
    # refs: https://qiita.com/everylittle/items/aa7c6f612ff0a9db7f01
    tmppath = os.path.join(tmpdir, remote_info)
    !git remote show origin > $tmppath
    with open(tmppath, 'r') as f:
        lines = [s.strip() for s in f.readlines()]
        
        # FIXME: GitHub等リポジトリにも対応するためハードコーディングを解消
        ssh_push_url = lines[2].replace('Push  URL: http://dg02.dg.rcos.nii.ac.jp/', 'ssh://root@dg02.dg.rcos.nii.ac.jp:3001/') 

api.siblings(action='add', name='gin', url=ssh_push_url)
clear_output()
print('SSH connection is ready.')

## 3. リポジトリ内のファイルを更新する

### 3-1. Git管理対象外ファイルを.gitignoreで設定する

In [ ]:
%%sh
#!/bin/bash

if [ ! -f ~/.gitignore ]; then  # .gitignoreファイルがなければ作成

echo "
/.cache/
/.conda/
/.config/
/.ipython/
/.local/
/.tmp/
/.bashrc
/.bash_logout
/.profile
/.netrc
.ipynb_checkpoints/
.fonts/
.jupyter/
.npm/
.ssh/
.jupyter-server-log.txt
.gitconfig
" >> ~/.gitignore

fi

###  3-2. 実験記録管理ファイルを作成する

In [ ]:
%%sh
#!/bin/bash
if [ ! -d ~/experiments/ ]; then
    # 実験配置用ディレクトリがなければ作成
    mkdir ~/experiments
fi

if [ ! -f ~/experiments/pipeline.json ]; then
    # pipeline.jsonがなければ初期化
    echo "[]" > ~/experiments/pipeline.json
fi

### 3-3 実験パッケージに必要なファイルを追加する

In [ ]:
%cd ~/

path_base_package = 'WORKFLOW/PACKAGE/base'

if not os.path.exists(path_base_package + '/.gitignore'):
    !cp .gitignore $path_base_package/.gitignore

if not os.path.exists(path_base_package + '/EX-WORKFLOW/param_files'):
    !mkdir -p $path_base_package/EX-WORKFLOW/param_files
    !cp WORKFLOW/FLOW/param_files/params.json $path_base_package/EX-WORKFLOW/param_files/params.json

if not os.path.exists(path_base_package + '/EX-WORKFLOW/util/base_datalad_save_push.ipynb'):
    !cp WORKFLOW/FLOW/util/base_datalad_save_push.ipynb $path_base_package/EX-WORKFLOW/util/base_datalad_save_push.ipynb

if not os.path.exists(path_base_package + '/Dockerfile'):
    !cp Dockerfile $path_base_package/Dockerfile

### 3-4. READMEに実行環境へのリンクを追加する

In [ ]:
import os
%cd ~/

# READMEに遷移リンクがあれば削除してから、新しく実行環境のリンクを追記する
path = 'README.md'
s = ''
with open(path, 'r') as f:
    s = f.read()
    s = s[:s.find('## ワークフロー実行が2回目以降の場合')]
    
with open(path, 'w') as f:
    f.write(s)

with open(path, 'a', newline='\n') as f:
    f.write("\n## ワークフロー実行が2回目以降の場合\nワークフロー実行環境へ遷移する場合は以下のリンクをクリックしてください<br>https://jupyter.cs.rcos.nii.ac.jp" + os.environ["JUPYTERHUB_SERVICE_PREFIX"] + "notebooks/WORKFLOW/FLOW/base_FLOW.ipynb")
    f.write("<br>※上記のリンクからワークフロー機能トップページに遷移できない場合は、「初期設定をする」のmaDMP遷移ボタンからワークフロー機能トップページに遷移してください。その場合maDMPの実行は不要です。")

## 4. 実行結果をデータガバナンス機能に同期する

ここまでの内容を保存し、データガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
# ノートブックを保存する
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
# SSHホスト（＝GIN）を信頼する設定
# ドメイン名がハードコーディングにつき要修正
with open('/home/jovyan/.ssh/config', mode='w') as f:
    f.write('host dg02.dg.rcos.nii.ac.jp\n\tStrictHostKeyChecking no\n\tUserKnownHostsFile=/dev/null\n')

In [ ]:
import papermill as pm

%cd ~/WORKFLOW/FLOW/util
pm.execute_notebook(
    './base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '[GIN] 研究リポジトリ初期設定を完了', TO_GIT=True)
)
print('データ同期が完了しました。')


## 5. ワークフロー図を更新する

ワークフロー図にこのワークフローが実行済みであることを反映します。

In [ ]:
%cd ~/
with open('.gitignore', 'r') as f:
    text = f.read()
    if text.find("notebooks.diag") == -1:
        !echo "/WORKFLOW/images/notebooks.diag" >> ./.gitignore

# notebooks.diagのgit管理を外す
!git update-index --skip-worktree WORKFLOW/images/notebooks.diag

In [ ]:
%cd ~/WORKFLOW/images

find = '"base_required_every_time"[fontsize = 14];'
replace = '"base_required_every_time"[numbered = 済, fontsize = 14];'
path = 'notebooks.diag'

with open(path, 'r') as f:
    s = f.read()

with open(path, 'w') as f:
    s = s.replace(find, replace)
    f.write(s)

## 6. ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../base_FLOW.ipynb)からトップページに遷移できます。  